In [2]:
import logging
import os
from http import HTTPStatus
from typing import Annotated, List, Dict
from uuid import UUID, uuid4
from operator import itemgetter
import boto3
from pathlib import Path

from langchain.chains.llm import LLMChain
from langchain_community.chat_models import ChatLiteLLM
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_elasticsearch import ApproxRetrievalStrategy, ElasticsearchStore
from elasticsearch import Elasticsearch
from langchain.schema import StrOutputParser
from langchain_core.runnables import Runnable, RunnableLambda, RunnablePassthrough, chain
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_elasticsearch import ElasticsearchRetriever

from redbox.model_db import MODEL_PATH
from redbox.models import EmbeddingModelInfo, Settings, ChatMessage, ChatResponse
from redbox.models.chat import ChatRequest, ChatResponse, SourceDocument, SelectedDocument
from redbox.storage import ElasticsearchStorageHandler
from core_api.ingest_runnables import make_worker_ingest_runnable, LocalFile

In [2]:

env = Settings(_env_file=".env")
env.elastic.host = "localhost"
env.minio_host = "localhost"

embedding_model = SentenceTransformerEmbeddings(model_name=env.embedding_model, cache_folder="../models/")

es = Elasticsearch(
    hosts=[
        {
            "host": "localhost",
            "port": env.elastic.port,
            "scheme": env.elastic.scheme,
        }
    ],
    basic_auth=(env.elastic.user, env.elastic.password),
)

if env.elastic.subscription_level == "basic":
    strategy = ApproxRetrievalStrategy(hybrid=False)
elif env.elastic.subscription_level in ["platinum", "enterprise"]:
    strategy = ApproxRetrievalStrategy(hybrid=True)

vector_store = ElasticsearchStore(
    es_connection=es,
    index_name="summarisation-chunk",
    embedding=embedding_model,
    strategy=strategy,
    vector_query_field="embedding",
)

print(os.environ["AZURE_API_VERSION"])
llm = ChatLiteLLM(
    model="azure/gpt-35-turbo",
    api_base="https://oai-i-dot-ai-playground-sweden.openai.azure.com/",
    api_version="2024-02-01",
    max_tokens=1024
)

storage_handler = ElasticsearchStorageHandler(es_client=es, root_index="summarisation")
s3_client = boto3

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2
/root/.cache/pypoetry/virtualenvs/redbox-ZGz5Jzj1-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
/root/.cache/pypoetry/virtualenvs/redbox-ZGz5Jzj1-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainPendingDeprecationWarning: The class `ApproxRetrievalStrategy` will be deprecated in a future version. Use DenseVectorStrategy instead.
  warn_deprecated(


2024-02-01


In [3]:
ingest = make_worker_ingest_runnable(
    storage_handler=storage_handler,
    s3_client=boto3.client(
        "s3", 
        endpoint_url=f"http://{env.minio_host}:{env.minio_port}", 
        aws_access_key_id=env.aws_access_key, 
        aws_secret_access_key=env.aws_secret_key
    ),
    env=env,
    chunk_size=1024
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2
/root/.cache/pypoetry/virtualenvs/redbox-ZGz5Jzj1-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu


In [ ]:
es.options(ignore_status=[400,404]).indices.delete(index="summarisation-file")
es.options(ignore_status=[400,404]).indices.delete(index="summarisation-chunk")

file = LocalFile(
    filepath=Path("../data/Conservative-Manifesto-GE2024.pdf"),
    creator_user_uuid=uuid4()
)
ingest.invoke(file)
print(file.creator_user_uuid)


In [27]:

creator_user_uuid = "5060f624-bb1b-43a9-ad88-885e1f226e38"
file_uuid=""

request = ChatRequest(
    message_history=[
        ChatMessage(role="system", text=system_prompt),
        ChatMessage(role="user", text="Are the conservatives going to cut national insurance?"),
        ChatMessage(role="ai", text="**Based on the information provided, the Conservatives are planning to cut national insurance**. They have already started cutting National Insurance taxes and their long-term ambition is to keep cutting it until it's gone. The next step in their plan is to cut employee National Insurance to 6% by April 2027, which would be a total tax cut of £1,350 for the average worker on £35,000. However, the document does not mention whether this cut will be implemented specifically for conservatives or if it is part of a broader policy."),
        ChatMessage(role="user", text=user_prompt)
    ],
    selected_files=[
        #SelectedDocument(file_uuid)
    ]
)

rag_chain = make_rag_runnable(
    llm,
    make_es_retriever(es, embedding_model, chunk_index_name="summarisation-chunk")
)

question = "When are they going to reduce it?"
r = rag_chain.invoke({
    "question": question,
    "file_uuids": request.selected_files,
    "user_uuid": creator_user_uuid,
    "chat_history": request.message_history
})

INFO:elastic_transport.transport:POST http://localhost:9200/summarisation-chunk/_search [status:200 duration:0.010s]
INFO:httpx:HTTP Request: POST https://oai-i-dot-ai-playground-sweden.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
15:07:51 - LiteLLM:INFO: utils.py:3397 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


In [28]:

print(r.output_text)
for s in r.source_documents:
    print(s)

**The Conservatives are planning to reduce national insurance in phases**. In the first half of this year, they have already begun cutting National Insurance taxes. Their long-term ambition is to keep cutting National Insurance until it's gone. The next step in their plan is to cut employee National Insurance to 6% by April 2027, which would be a total tax cut of £1,350 for the average worker on £35,000.
page_content='In the first half of this year, we began cutting National Insurance. It is unfair that working people pay two taxes on their income – income tax and National Insurance – when other people only pay income tax.\n\nOur long-term ambition, when it is affordable to do so, is to keep cutting National Insurance until it’s gone, as part of our plan to make the tax system simpler and fairer. As the next step in that plan, we will cut employee National Insurance to 6% by April 2027 – meaning that we will have halved it from 12% at the beginning of this year, a total tax cut of £1,3

In [22]:
print(r['prompt'].to_string())

System: 
Given the following extracted parts of a long document and 
a question, create a final answer. 
If you don't know the answer, just say that you don't know. Don't try to make up an answer. 
If a user asks for a particular format to be returned, such as bullet points, then please use that format. 
If a user asks for bullet points you MUST give bullet points. 
If the user asks for a specific number or range of bullet points you MUST give that number of bullet points. 

Use **bold** to highlight the most question relevant parts in your response. 
If dealing dealing with lots of data return it in markdown table format. 

Human: Are the conservatives going to cut national insurance?
AI: **Based on the information provided, the Conservatives are planning to cut national insurance**. They have already started cutting National Insurance taxes and their long-term ambition is to keep cutting it until it's gone. The next step in their plan is to cut employee National Insurance to 6% by Ap